<a href="https://colab.research.google.com/github/nikitaoltyan/Mech-Trading-Systems/blob/main/AROON_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Project cloning
!git clone https://github.com/nikitaoltyan/Mech-Trading-Systems.git
%cd Mech-Trading-Systems

Cloning into 'Mech-Trading-Systems'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 283 (delta 10), reused 3 (delta 1), pack-reused 262
Receiving objects: 100% (283/283), 1.78 MiB | 7.30 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/Mech-Trading-Systems


In [31]:
import pandas as pd
import numpy as np
from PlotDraw import draw_plot

In [7]:
data = pd.read_excel("LHAG.xlsx")
data.head()

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M


In [35]:
data.astype({'<CLOSE>': 'int32', '<OPEN>': 'int32', '<HIGH>': 'int32', '<LOW>': 'int32'}).dtypes
data.astype({'<DATE>': 'datetime64[ns]'}).dtypes

<DATE>     datetime64[ns]
<CLOSE>           float64
<OPEN>            float64
<HIGH>            float64
<LOW>             float64
<VOL>              object
dtype: object

In [9]:
draw_plot(data, 0, len(data))

In [10]:
! pip install mpl_finance

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as plt_dates
import matplotlib.ticker as plt_ticker

In [45]:
def calc_aroon_indicator(data, col_names, periods = 14):
    aroon_data = data.copy()
    aroon_up = data['<HIGH>'].rolling(periods+1).apply(lambda x: x.argmax(), raw=True) / periods * 100
    aroon_down = data['<LOW>'].rolling(periods+1).apply(lambda x: x.argmin(), raw=True) / periods * 100
    aroon_data[col_names[0]] = aroon_up
    aroon_data[col_names[1]] = aroon_down
    return aroon_data

In [41]:
col_names = ['Aroon_Up', 'Aroon_Dn']
aroon_LHAG = calc_aroon_indicator(data, col_names, periods=30)
aroon_LHAG

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,Aroon_Up,Aroon_Dn
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,NaN,NaN
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,NaN,NaN
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,NaN,NaN
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,NaN,NaN
...,...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,100.000000,26.666667
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,100.000000,23.333333
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,100.000000,20.000000
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,96.666667,16.666667


In [30]:
money = 100000
is_purchased = False
last_price = 0
current_shares = 0
commission = 0.003

for _, row in aroon_LHAG.iterrows():
    buy_condition = (row["Aroon_Up"]>70) and (row["Aroon_Dn"])
    sell_condition = (row["Aroon_Up"]<row["Aroon_Dn"])
    if (buy_condition) and not (is_purchased):
        is_purchased = True
        open_price = row["<CLOSE>"]
        current_shares = money // open_price
        deal_commission = round(current_shares * open_price * commission, 5)
        money = money - (open_price * current_shares) - deal_commission
    if (sell_condition) and (is_purchased):
        is_purchased = False
        close_price = row["<CLOSE>"]
        deal_commission = round(current_shares * close_price * commission, 5)
        money = money + (close_price * current_shares) - deal_commission
        current_shares = 0
    last_price = row["<CLOSE>"]
print(money+ current_shares*last_price)

218122.64860000016


In [32]:
def calculate_profit(money, period):
    aroon_LHAG = calc_aroon_indicator(data, col_names, periods=period)
    money = money
    is_purchased = False
    last_price = 0
    current_shares = 0
    commission = 0.003

    for _, row in aroon_LHAG.iterrows():
        buy_condition = (row["Aroon_Up"]>70) and (row["Aroon_Dn"])
        sell_condition = (row["Aroon_Up"]<row["Aroon_Dn"])
        if (buy_condition) and not (is_purchased):
            is_purchased = True
            open_price = row["<CLOSE>"]
            current_shares = money // open_price
            deal_commission = round(current_shares * open_price * commission, 5)
            money = money - (open_price * current_shares) - deal_commission
        if (sell_condition) and (is_purchased):
            is_purchased = False
            close_price = row["<CLOSE>"]
            deal_commission = round(current_shares * close_price * commission, 5)
            money = money + (close_price * current_shares) - deal_commission
            current_shares = 0
        last_price = row["<CLOSE>"]
    return money+current_shares*last_price

In [46]:
periods = [1, 30]
start_money = 100000
results = {}

for iter in range (1024):
    period = np.random.randint(periods[0], periods[1])
    profit = calculate_profit(start_money, period)
    
    results[profit-start_money] = period
    print(f"Iter: {iter}, profit: {profit-start_money}, dif: {period}")

print(f"Max profit: {max(results)} for $100k start money was received. \nUsed period: {results[max(results)]}")

Iter: 0, profit: 110747.37682, dif: 28
Iter: 1, profit: 110747.37682, dif: 28
Iter: 2, profit: 91472.11469000002, dif: 24
Iter: 3, profit: 168216.3931799999, dif: 9
Iter: 4, profit: 351228.0996200003, dif: 4
Iter: 5, profit: 168216.3931799999, dif: 9
Iter: 6, profit: 492026.03449999995, dif: 2
Iter: 7, profit: 348034.46869999997, dif: 6
Iter: 8, profit: 492026.03449999995, dif: 2
Iter: 9, profit: 207335.54799000017, dif: 11
Iter: 10, profit: 110747.37682, dif: 28
Iter: 11, profit: 348034.46869999997, dif: 6
Iter: 12, profit: 103654.45412999994, dif: 26
Iter: 13, profit: 118122.64860000016, dif: 14
Iter: 14, profit: 171679.94299999997, dif: 7
Iter: 15, profit: 112234.28448999999, dif: 29
Iter: 16, profit: 110747.37682, dif: 28
Iter: 17, profit: 206715.75176999997, dif: 10
Iter: 18, profit: 348034.46869999997, dif: 6
Iter: 19, profit: 384190.29317000014, dif: 3
Iter: 20, profit: 206715.75176999997, dif: 10
Iter: 21, profit: 236369.96280000027, dif: 12
Iter: 22, profit: 171679.94299999997